# The Ultimate Guide to the DebateAI Frontend

This notebook is your all-in-one guide to the DebateAI frontend. We’ll go from a 10,000-foot view of the architecture right down to the nitty-gritty of how individual components and functions work. Whether you’re looking to understand the project, fix a bug, or add a new feature, this is the place to start.

## 1. The Big Picture: Architecture and Data Flow

Before we dive into the code, let’s understand how the application is structured and how data moves through it.

### In Layman’s Terms

Imagine the app is a restaurant.

- **The User** is the customer.
- **The UI (React Components)** is the dining room—what the customer sees and interacts with (tables, menus, etc.).
- **The Brains (React Hooks, especially `useDebate`)** is the restaurant manager. The manager keeps track of everything: what the customer ordered, what the kitchen is cooking, and when the food is ready.
- **The Kitchen (Backend API)** is where the food is made. The manager sends orders to the kitchen and gets food (data) back.
- **The Waiter (API Client — `debateClient.ts`, `auth/api.ts`)** is responsible for taking the manager’s orders to the kitchen and bringing the food back.
- **The Doorman (`ProtectedRoute`)** checks if the customer has a reservation (is logged in) before letting them into the main dining area.

### The Technical Flow

Here’s a more technical breakdown of the data flow, from user login to watching a debate:

```
1. User visits the app.
   |
   +--> React App loads (`main.tsx`).
   |     |
   |     +--> AuthProvider wraps the app, checks for a cookie (`authProvider.tsx`).
   |           |
   |           +--> Calls getMe() to the backend (`auth/api.ts`).
   |
2. User navigates to a page (e.g., `/login` or `/debate`).
   |
   +--> React Router (`Routes/Routes.tsx`) directs them.
   |     |
   |     +--> If going to `/debate`, ProtectedRoute checks `useAuth()`'s `isAuthenticated` flag.
   |           |
   |           +--> If not authenticated, redirects to `/login`.
   |
3. User logs in (`LoginPage.tsx`).
   |
   +--> Calls `login()` from `useAuth()` context.
   |     |
   |     +--> `authProvider.tsx` calls `postLogin()` to the backend.
   |           |
   |           +--> Backend returns a session cookie.
   |           +--> `authProvider` then calls `refresh()` to get user data and sets `isAuthenticated = true`.
   |
4. User is on the Debate Page (`DebatePage.tsx`).
   |
   +--> This page uses the `useDebate` hook (`hooks/useDebate.ts`).
   |     |
   |     +--> User starts a debate.
   |     |
   |     +--> `useDebate` calls a streaming function (e.g., `versusStartStream`) from `api/debateClient.ts`.
   |           |
   |           +--> This makes a POST request to the backend, which keeps the connection open.
   |
5. Backend sends data chunks (NDJSON events).
   |
   +--> `debateClient.ts` receives the events.
   |     |
   |     +--> It calls the `onEvent` callback function provided by `useDebate`.
   |           |
   |           +--> `useDebate`'s `onEvent` function processes the event (e.g., 'turn', 'delta').
   |                 |
   |                 +--> It updates the `messages` state using `setMessages()`
   |
6. React re-renders the UI.
   |
   +--> The `MessageList` component receives the new `messages` array and displays the updates.
```


## 2. Deep Dive: The Authentication Flow

Authentication is cookie-based. This means once you log in, the browser stores a special cookie that it sends with every subsequent request to the backend, proving who you are.

### How It Works, Step-by-Step

1.  **The Provider (`authProvider.tsx`):** The entire app is wrapped in `<AuthProvider>`. This component holds all the authentication logic and state (`user`, `isAuthenticated`, `isLoading`).

2.  **Initial Load:** When the app first loads, `AuthProvider`'s `useEffect` hook runs. It calls the `refresh()` function.

3.  **Checking the Session (`refresh()`):** The `refresh` function calls `getMe()` from `auth/api.ts`. This sends a request to the `/auth/me` endpoint on the backend. Because our API calls use `credentials: 'include'`, the browser automatically sends the session cookie if it exists.

4.  **Backend Response:**
    *   If the cookie is valid, the backend returns the user's data.
    *   If there's no cookie or it's invalid, the backend returns an error (like 401 Unauthorized).

5.  **State Update:** `AuthProvider` updates its state based on the response. If user data comes back, `isAuthenticated` becomes `true`. If not, it becomes `false`. The `isLoading` state is set to `false` once this check is complete.

6.  **Guarding Routes (`ProtectedRoute.tsx`):** When you try to access a page like `/debate`, the `ProtectedRoute` component checks the `isAuthenticated` and `isLoading` flags from `useAuth()`.
    *   If `isLoading`, it shows a loading message.
    *   If not `isAuthenticated`, it redirects you to `/login`.
    *   If `isAuthenticated`, it lets you through.

### How to Code It: The `useAuth` Hook

Anywhere in the app, you can get the current user's status like this. This is the power of the Provider pattern.

```typescript
import { useAuth } from '../auth/authProvider';

function MyComponent() {
  const { user, isAuthenticated, login, logout } = useAuth();

  if (!isAuthenticated) {
    return <button onClick={() => login('user', 'pass')}>Log In</button>;
  }

  return (
    <div>
      <p>Welcome, {user?.identifier}!</p>
      <button onClick={() => logout()}>Log Out</button>
    </div>
  );
}
```


## 3. The Heart of the App: The `useDebate` Hook

The `useDebate` hook (`src/hooks/useDebate.ts`) is where the magic happens. It's a complex piece of logic that manages the entire lifecycle of a debate.

### What It Does

-   **Manages State:** Holds all the important data for a debate: the list of messages, the current user input, loading states, and who is currently "typing".
-   **Handles User Actions:** Contains the functions that are called when a user starts a debate (`start()`) or sends a message (`send()`).
-   **Processes Server Events:** Listens to the stream of data from the backend and updates the state accordingly.
-   **Manages Sessions:** Saves the debate progress to local storage and allows you to resume a session later.

### The Inner Workings: Handling a Stream of Events

The most complex part of `useDebate` is handling the real-time data stream.

1.  **Initiating the Stream:** When `start()` or `send()` is called, it uses a function from `debateClient.ts` (e.g., `versusStartStream`). This function makes a `fetch` request but, crucially, passes an `onEvent` callback function.

2.  **The `onEvent` Callback:** This function lives inside `useDebate`. It's the designated handler for all events that come from the server for a specific debate.

3.  **The `switch` Statement:** The `onEvent` function uses a `switch` statement to handle different event types (`session`, `turn`, `delta`, `endturn`).

    *   `case 'turn'`: A new character is about to speak. The hook adds a new, empty message bubble to the `messages` array and sets `typingSpeaker` to show the typing indicator.
    *   `case 'delta'`: A small piece of text (a "delta") has arrived for the current turn. The hook finds the corresponding message in the `messages` array and appends the delta to its text.
    *   `case 'endturn'`: The character has finished speaking. The hook sets `typingSpeaker` to `null` to hide the indicator.

### How to Code It: The Typing Animation Logic

The smooth typing animation you see is a great example of the hook's sophistication. Here's a layman's explanation of the logic from the `README.md` and the code:

**The Goal:** When a character starts talking, we want a small delay before the first words appear, so the user has time to see who is talking. But we don't want to lose any text that arrives during that delay.

**The Code (`appendDelta` function in `useDebate.ts`):**

1.  **Is this the first text for this turn?** It checks a `Set` called `firstDeltaSeenRef` to see if we've already shown text for this turn.

2.  **If YES (it's the first):**
    *   It doesn't show the text immediately. Instead, it stores the text in a temporary `initialDeltaBufferRef`.
    *   It starts a `setTimeout` for `250ms`.

3.  **While the timer is ticking:** If more text arrives for the same turn, it just keeps adding it to the buffer.

4.  **When the timer finishes:**
    *   It takes everything that's in the buffer and appends it to the message bubble all at once.
    *   **Crucially, it now marks the turn as "seen"** in `firstDeltaSeenRef`.

5.  **If NO (it's not the first text):** It just appends the text to the message bubble immediately, because the initial delay is over.

This prevents the race condition where fast-arriving text chunks would render out of order.


## 4. How to Extend the App: Practical Guides

Let's put this knowledge into practice.

### How to Add a New Page and Route

Let's say you want to add a new `/profile` page.

1.  **Create the Page Component:** Create a new file `src/pages/ProfilePage.tsx`.

```typescript
// src/pages/ProfilePage.tsx
import { useAuth } from '../auth/authProvider';

export default function ProfilePage() {
  const { user } = useAuth();
  return (
    <div>
      <h1>Profile</h1>
      <p>Username: {user?.identifier}</p>
    </div>
  );
}
```

2.  **Add it to the Router:** Open `src/Routes/Routes.tsx`.

```typescript
// src/Routes/Routes.tsx

// ... other imports
const ProfilePage = lazy(() => import('../pages/ProfilePage')); // 1. Import it lazily

export default function AppRoutes() {
  return (
    <Suspense fallback={<div>Loading...</div>}>
      <Routes>
        {/* ... other routes */}

        {/* Protected branch */}
        <Route element={<ProtectedRoute />}>
          <Route path="/debate" element={<DebatePage />} />
          <Route path="/profile" element={<ProfilePage />} /> {/* 2. Add the route here */}
        </Route>

        {/* ... other routes */}
      </Routes>
    </Suspense>
  );
}
```

3.  **Add a Link (Optional):** You could add a link to it in `TopNav.tsx` or another component.

```tsx
import { Link } from 'react-router-dom';

<Link to="/profile">My Profile</Link>
```

And that's it! You've added a new, protected page to the application.


### How to Add a New API Call

Let's say you want to add a button that fetches a list of past debate sessions from the backend.

**1. Define the API function in `src/api/debateClient.ts`:**

First, you need to add a function that knows how to call the backend endpoint. We'll add a `listSessions` function.

```typescript
// src/api/debateClient.ts

// ... other functions

export async function listSessions() {
  const res = await fetch(`${API_BASE}/debate/sessions`, withCreds());
  if (!res.ok) throw new Error(await res.text());
  return res.json();
}
```

**2. Use the function in a component:**

Now, you can create a new component that uses this function to fetch and display the sessions.

```typescript
// src/components/SessionList.tsx

import React, { useState } from 'react';
import { listSessions } from '../api/debateClient';

export function SessionList() {
  const [sessions, setSessions] = useState<any[]>([]);
  const [isLoading, setIsLoading] = useState(false);

  const handleFetchSessions = async () => {
    setIsLoading(true);
    try {
      const sessionData = await listSessions();
      setSessions(sessionData);
    } catch (error) {
      console.error('Failed to fetch sessions:', error);
    } finally {
      setIsLoading(false);
    }
  };

  return (
    <div>
      <button onClick={handleFetchSessions} disabled={isLoading}>
        {isLoading ? 'Loading...' : 'Load Past Sessions'}
      </button>
      <ul>
        {sessions.map((session) => (
          <li key={session.id}>{session.title}</li>
        ))}
      </ul>
    </div>
  );
}
```

**3. Add the component to a page:**

Finally, you can add your new `SessionList` component to an existing page, like `DebatePage.tsx` or your new `ProfilePage.tsx`.

```typescript
// In one of your page components

import { SessionList } from '../components/SessionList';

// ... inside the page's return statement
<SessionList />
```

This three-step pattern (API function -> Component logic -> UI display) is a robust way to add new features that interact with the backend.


## 5. The Component Library & Styling

This project uses a combination of custom components and **Tailwind CSS** for styling. This is a modern and highly efficient way to build user interfaces.

### The Philosophy: Utility-First CSS

Instead of writing traditional CSS files (e.g., `App.css`), Tailwind works by providing thousands of small, single-purpose utility classes that you apply directly in your HTML (or JSX).

- **Instead of this:** `<div class=\"user-card\">...</div>` (and then defining `.user-card` in a CSS file).
- **You do this:** `<div class=\"p-4 bg-white rounded-lg shadow-md\">...</div>`.

This approach has several advantages:
- **Speed:** You rarely need to leave your component file to write styles.
- **Consistency:** It's easier to maintain a consistent design system (spacing, colors, etc.).
- **Performance:** Tailwind automatically removes all unused CSS classes when you build for production, resulting in a very small final CSS file.

### Component Showcase: `CharacterCard.tsx`

Let's look at `src/components/CharacterCard.tsx` as an example of how these concepts come together.

**What it is:** A card that displays a character's portrait, name, and quote. It's used in the `CharacterSelector` to allow the user to pick who will participate in the debate.

**How it's Built:**

```typescript
// src/components/CharacterCard.tsx (annotated)

import { motion } from "framer-motion"; // For animations
import type { CharData } from "../data/CharData";

export default function CharacterCard({
  character, // The data for the character to display
  isSelected, // Is this card currently selected?
  disabled,   // Is this card disabled?
  onSelect,   // Function to call when the card is clicked
}: {
  character: CharData;
  isSelected?: boolean;
  disabled?: boolean;
  onSelect?: () => void;
}) {
  return (
    <motion.button // It's a button, and it's animated!
      // ...props for accessibility and event handling
      className={[
        // An array of strings is used to build the final className
        // This makes it easy to conditionally apply classes

        // Base styles
        "group relative w-full h-40 sm:h-44 md:h-48 overflow-hidden rounded-2xl border transition",
        "flex flex-col justify-end text-left",
        "bg-neutral-900 border-neutral-700",

        // Conditional styles for disabled state
        disabled
          ? "opacity-50 cursor-not-allowed"
          : "hover:shadow-lg focus:outline-none",

        // Conditional styles for selected state
        isSelected
          ? "ring-2 ring-amber-400 border-amber-400"
          : "ring-1 ring-transparent hover:border-amber-400/50",
      ].join(" ")}
      onClick={disabled ? undefined : onSelect}
      disabled={disabled}
    >
      {/* The character's portrait image */}
      <img src={character.image} alt={character.name} className="absolute inset-0 w-full h-full object-cover opacity-80 group-hover:opacity-100 transition" />

      {/* A decorative glow effect that appears on hover */}
      <div className="pointer-events-none absolute inset-0 bg-gradient-to-t from-black/70 to-transparent opacity-100" />

      {/* The text content (name and quote) that appears on hover */}
      <motion.div
        initial={{ opacity: 0, y: 8 }}
        animate={{ opacity: 1, y: 0 }}
        transition={{ duration: 0.25 }}
        className="relative z-10 p-4"
      >
        <h3 className="text-white font-semibold text-lg">{character.name}</h3>
        <p className="text-neutral-300 text-sm">“{character.quote}”</p>
      </motion.div>
    </motion.button>
  );
}
```

**Key Takeaways:**

- **Conditional Styling:** The component dynamically changes its appearance based on the `isSelected` and `disabled` props. This is done using ternary operators inside the `className` array.
- **Group Hover:** Tailwind's `group` feature is used to show child elements (like the character's name and quote) when the parent button is hovered over. This is a very powerful feature that keeps the logic within the component.
- **Animation with Framer Motion:** The component uses the `motion` library to add subtle animations, like scaling on tap (`whileTap`) and fading in the text (`initial`, `animate`). This adds a layer of polish to the user experience.
